## Kelime-Bazlı Tokenizer Pipeline — Teorik Özeti

### 1. Temizleme & Normalizasyon  
- **Unicode Normalizasyon** (`NFC`) ve **küçültme**: Metindeki tüm harfleri standart forma getirir, büyük/küçük farkını ortadan kaldırır.  
- **Regex Filtreleme**: Sadece Türkçe karakterler, sayılar ve boşluklar korunur; geri kalan tüm işaretler boşluğa dönüştürülür.  

### 2. Konuşmacı & Stil Etiketleri  
- **`<user>` / `<bot>`**: Diyalog metnindeki rol bilgisini netleştirir.  
- **`<casual>`** (veya `<formal>`): Modelin hangi tonda cevap vereceğini belirtir.  

Metin başına eklenen bu token’lar, modelin “kimin” ve “hangi tarzda” konuştuğunu öğrenmesine imkân tanır.

### 3. Sıra Kontrol Token’ları  
- **`<sos>` / `<eos>`**: Decoder’a “başla” ve “bitir” sinyalleri gönderir.  
- **Padding (`0` ID’si)**: Tüm diziler aynı uzunluğa getirilir; model sabit boyutlu tensörlerle çalışır.

### 4. Frekans-Bazlı Sözlük İnşası  
- **Geçici Tokenizer** ile tüm korpustaki token frekansları sayılır.  
- **`min_freq` Filtresi**: Çok nadir (örn. yalnızca 1 kez) görülen kelimeler sözlük dışına alınır → gürültü azalır.  
- **Özel Token’ların Eklenmesi**: `<user>`, `<bot>`, `<casual>`, `<unk>`, `<sos>`, `<eos>` kesinlikle sözlükte kalır.

### 5. Final Tokenizer & Dönüşüm  
- Sabitlenen `keep_tokens` listesiyle **`Tokenizer.fit_on_texts`** çağrıları yapılır.  
- Metinler, **`texts_to_sequences`** ile ID dizilerine; ardından **`pad_sequences`** ile eşit uzunluğa dönüştürülür.  

### 6. Dynamic UNK-Dropout  
- Eğitim sırasında **rastgele** belli oranla (örn. `%5–12`) gerçek token’lar **`<unk>`** (OOV) ile değiştirilir.  
- **Epoch takvimi** ile dropout oranı  
  - Erken dönemde düşük (%5) → temel yapıyı öğrenme  
  - Orta dönemde yüksek (%12) → genelleme  
  - Son dönemde orta seviyeye (%8) → ince ayar  
- Model, eksik veya bilinmeyen kelimelere karşı daha dayanıklı hâle gelir.

### 7. `tf.data.Dataset` Pipeline  
- **`.map(unk_dropout)`** → her batch’te augmentasyon  
- **`.shuffle()`** → her epoch veriyi karıştırma  
- **`.batch(drop_remainder=True)`** → sabit batch boyutu  
- **`.prefetch()`** → GPU/CPU paralelliği  


Bu adımlar bir araya geldiğinde, **küçük ve orta ölçekli Türkçe chatbot** projeleri için hafif, esnek ve üretime hazır bir tokenizer hattı elde edilmiş olur—dış bağımlılıksız, okunabilir ve kolayca ayarlanabilir. ```


In [4]:
import re
import unicodedata
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [5]:
FILTERS = ""
OOV_TOKEN = "<unk>"
SOS_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"
CHAR_LEVEL = False
MİN_FREQ = 1
MAX_VOCAB = None
PADDING = "post"

USER_TOKEN , BOT_TOKEN , STYLE_TOKEN = "<user>" , "<bot>" , "<casual>"


In [6]:
df = pd.read_csv("örnek_set .csv")
df.head()

,input,output
0,Merhaba,"Merhaba, size nasıl yardımcı olabilirim?"
1,Nasılsın?,"İyiyim, teşekkür ederim. Siz nasılsınız?"
2,Adın ne?,Ben bir yapay zekâ asistanıyım. Adım yok ama y...
3,Kaç yaşındasın?,"Benim yaşım yok, dijitalim!"
4,Bugün günlerden ne?,"Maalesef tarih bilgim yok, ama sistem saatinde..."


In [7]:
raw_input = df["input"].astype(str).to_list()
raw_targets = df["output"].astype(str).to_list()

In [8]:
def clean_texts(text:str)->str:
    text = unicodedata.normalize("NFC",text).lower()
    text = re.sub(r"[^a-zçğıöşüğ0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

In [9]:
inputs = [f"{USER_TOKEN} {STYLE_TOKEN} {clean_texts(t)}" for t in raw_input]
targets = [f"{BOT_TOKEN} {STYLE_TOKEN} {clean_texts(t)}" for t in raw_targets]

In [ ]:
dec_in = [f"{SOS_TOKEN} {t}" for t in targets]
dec_out = [f"{t} {EOS_TOKEN}" for t in targets]
all_texts = inputs + dec_in + dec_out

In [11]:
'''
İlk geçişte tüm metinleri tarayıp her token’ın kaç kez göründüğünü sayıyoruz
Böylece hangi kelimelerin “nadiren” (örneğin yalnızca 1 kez) geçtiğini görebiliriz
'''

# ── 3) Geçici tokenizer → frekans tablosu ─────────────────────
tmp_tok = Tokenizer(filters=FILTERS, oov_token=OOV_TOKEN, char_level=CHAR_LEVEL)
tmp_tok.fit_on_texts(all_texts)
word_counts = tmp_tok.word_counts   # {token: count}

In [12]:
keeps_token = [w for w,c in word_counts.items() if c >= MİN_FREQ]

for sp in (USER_TOKEN,BOT_TOKEN,STYLE_TOKEN,OOV_TOKEN,SOS_TOKEN,EOS_TOKEN):
    if sp not in keeps_token:
        keeps_token.append(sp)

In [13]:
keeps_token

['<user>',
 '<casual>',
 'merhaba',
 'nasılsın',
 'adın',
 'ne',
 'kaç',
 'yaşındasın',
 'bugün',
 'günlerden',
 'hangi',
 'dilleri',
 'konuşuyorsun',
 'bana',
 'bir',
 'şaka',
 'yap',
 'seni',
 'kim',
 'yaptı',
 'işe',
 'yararsın',
 'telefon',
 'numaran',
 'nedir',
 'hava',
 'nasıl',
 'seviyorum',
 'sıkıldım',
 'film',
 'öner',
 'yemek',
 'yapayım',
 'benimle',
 'konuş',
 'yalnızım',
 'uyuyamıyorum',
 'biraz',
 'müzik',
 'aç',
 'şiir',
 'oku',
 'sence',
 'hayat',
 'arkadaşın',
 'var',
 'mı',
 'oyun',
 'oyna',
 'sen',
 'zeki',
 'misin',
 'gece',
 'mi',
 'gündüz',
 'mü',
 'yarın',
 'olacak',
 'yazılım',
 'bilgisayar',
 'i',
 'nterneti',
 'buldu',
 'gerçek',
 'robot',
 'musun',
 'en',
 'iyi',
 'programlama',
 'dili',
 'hangisi',
 'kodlama',
 'öğrenmek',
 'zor',
 'mu',
 'duygusal',
 'mısın',
 'yapay',
 'zek',
 'makine',
 'öğrenmesi',
 'veri',
 'bilimi',
 'python',
 'java',
 'tarih',
 'rastgele',
 'bilgi',
 'ver',
 'sevilen',
 'uyumalı',
 'mıyım',
 'motive',
 'olabilirim',
 'hikaye',
 'anl

In [14]:
if MAX_VOCAB:
    keeps_token = sorted(
        keeps_token,
        key = lambda w :word_counts.get(w,0),
        reverse=True
        )[:MAX_VOCAB]
    
    '''
Sıralama: En yüksek frekanslı kelimeler öncelikli.

Dilime Alma [:MAX_VOCAB]: Sadece ilk N kelime kalır.

Böylece embed tablonuzun boyutu doğrudan MAX_VOCAB ile sınırlanır, geri kalan her şey eğitimin tamamında <unk> olarak temsil edilir.
    
    '''

In [15]:
Tokenizer = Tokenizer(filters=FILTERS , oov_token=OOV_TOKEN, char_level= CHAR_LEVEL,num_words=len(keeps_token)+1)

Tokenizer.fit_on_texts(keeps_token)
Tokenizer.fit_on_texts(all_texts)

In [16]:
inp_seq = Tokenizer.texts_to_sequences(inputs)
dec_in_seq = Tokenizer.texts_to_sequences(dec_in)
dec_out_seq = Tokenizer.texts_to_sequences(dec_out)

max_len = max(map(len , inp_seq + dec_in_seq + dec_out_seq))
print(max_len)

14


In [17]:
encoder_input = pad_sequences(inp_seq,padding=PADDING,value=0,maxlen=max_len)
decoder_input = pad_sequences(dec_in_seq,padding=PADDING,value=0,maxlen=max_len)
decoder_output = pad_sequences(dec_out_seq,padding=PADDING,value=0,maxlen=max_len)

In [18]:
coverage = sum(word_counts.get(w, 0) for w in keeps_token) / sum(word_counts.values())
print(f"Vocabulary coverage (min_freq={MİN_FREQ}): {coverage:.2%}")

Vocabulary coverage (min_freq=1): 100.00%


In [19]:
unk_id = Tokenizer.word_index.get(OOV_TOKEN)
drop_var = tf.Variable(0.05,trainable=False,dtype= tf.float32)

print("unk_id ==" ,unk_id)

def unk_dropout(inputs,targets):
    e = inputs["encoder_input"]
    d = inputs["decoder_input"]

 # maskeler bool olmalı
    mask_e = tf.random.uniform(tf.shape(e), dtype=tf.float32) < drop_var
    mask_d = tf.random.uniform(tf.shape(d), dtype=tf.float32) < drop_var

    inputs["encoder_input"] = tf.where(mask_e, unk_id, e)
    inputs["decoder_input"] = tf.where(mask_d, unk_id, d)
    return inputs, targets

unk_id == 316


In [20]:
def schedule(epoch):
    if epoch < 5: return 0.05
    if epoch < 15 : return 0.12

    return 0.08

In [21]:
class UnkSchedule(tf.keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs=None):
        rate = schedule(epoch)
        drop_var.assign(rate)
        print(f"[UNK-Dropout] epoch {epoch} → rate={rate:.2f}")

### Modeli bu pipeline ile eğitmek için:
#### model.fit(dataset, epochs=20, callbacks=[UnkScheduler()])

In [22]:
batch_size = 32
buffer_size = 1000

In [23]:
dataset = (
    tf.data.Dataset.from_tensor_slices((
        {"encoder_input": encoder_input, 
         "decoder_input": decoder_input
        },
         decoder_output)
    )
    .map(unk_dropout, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size)
    .batch(batch_size, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)


---
----

## Bazı kodların ne anlama geldiğine gelin biraz daha bakalım.Kısa fonksiyonlar ya da bakıldığında soru işareti içerebilecek sorunları ortadan kaldırmaya çalışalım.

----
-----

* Önce frame i ekleyelim.
* Sonra clean_texts ile frame i temizleyelim.
* Sonra frame i input ve targets olarak 2 parçaya ayıralım.
* Sonra bot user ve casual etiketlerini geçirelim.
* Sonra sos ve eos tokenlarını projeye entegre edelim.
* Geçici tokenizer i eğittikden sonra min_freq uygulamasına geçelim.
* Sonrasında max_vocab ile sınırlandırmaları belirleyelim.
* Sonrasında fit_on_texts ve texts_to_sequences fonklarıyla tokenizer i eğitelim
* Sonrasında maxlen ve padding işlemlerini yapalım.
* Sonrasında unk_dropout işlemlerini yapalım ve dataset yolunu oluşturalım.

---
---

### Keeps tokens i inceleyelim.Bakalım burada yapılan işlemlerin amacı neymiş.


In [25]:
keeps_token = [w for w,c  in word_counts.items() if c>= MİN_FREQ]

for sp in (USER_TOKEN,BOT_TOKEN,SOS_TOKEN,EOS_TOKEN,OOV_TOKEN,STYLE_TOKEN):
    if sp not in keeps_token:
        keeps_token.append(sp)

* Burada yapılan işlem keeps token değişkenindeki c sabiti eğer min_freq ( min frekans ) değerinden büyük ve eşit ise çalışıyor.Yani keeps token içerisinde bulunan word_counts değişkenindeki her kelime önce min freq ile kıyaslanıyor ve eğer min_freq değerinden büyükse işleme alınıyor.Sp denilen değişken ise eğer keeps_token içerisindeki değerlere sahip değilse elle ekleniyor.Aslında temel mantık min_freq işleminine göre kelime bazlı kontrol yapılıyor.

---
---

### Max_Vocab değerini ölçelim ve bunu neden işleme sokuyoruz tartışalım...

In [29]:
if MAX_VOCAB:
    keeps_token = sorted(
           keeps_token,
           key = lambda w: word_counts.get(w,0),
           reverse = True        
                   )[:MAX_VOCAB]
    
    '''
    reverse=True

Bu sayede en yüksek frekansa sahip kelimeler listenin başına gelir (azalan düzende sıralama).
    
    '''

    '''
    key=lambda w: word_counts.get(w,0)

sorted() fonksiyonuna verdiğiniz bu key parametresi, her öğe (w) için hangi değere göre sıralama yapacağınızı tanımlar.
    '''

* Keeps tokenin içerisindeki değerler tek tek en büyük ve en küçük sıralamasına sokuluyor.Keepss token üzerinden işlem yapılarak bir key oluşturuluyor.Bu key her w için hangi değere göre sıralama yapacağını belitir.Reverse parametresi en büyük değeri listenin başına yerleştirir.Ve bu işlemler sıralamadan sonra listenin ilk MAX_VOCAB elemanını alır.Yani keeps_token değişkeni artık en sık geçen ilk MAX_VOCAB kelimeyi içerir

-----
-----

### Son olarak unk_dropout kısmına göz gezdirelim.

In [30]:
unk_id = Tokenizer.word_index.get(OOV_TOKEN)
drop_var = tf.Variable(0.05,trainable=False,dtype = tf.float32)

def unk_dropout(inputs,targets):
    e = inputs["encoder_input"]
    d = inputs["decoder_input"]

    mask_e = tf.random.uniform(tf.shape(e) , dtype = tf.float32) < drop_var  
    mask_d = tf.random.uniform(tf.shape(d) , dtype = tf.float32) < drop_var

    inputs["decoder_input"] = tf.where(mask_d,unk_id ,d)
    inputs["encoder_input"] = tf.where(mask_e,unk_id,e)

    return targets,inputs

* Önce OOV_TOKEN değerinin id numarasını unk_id değişkenine atadık.Drop var değişkeni yüzde 5 den başlayarak eğitilmeden devam edecek.UNK_DROPOUT fonksiyonunda ise birazdan tanımlayacağımız daha doğrusu parametre olarak alacağımız değerleri kullanacağız.Bu çıktının temel amacı belirli yüzdeliklerde bulunan değerlerle epoch sayısına oran bağlamak.Diğer fonksiyona geçelim.Çok daha rahat anlayacaksınız.

In [31]:
def schedule(epoch):
    if epoch < 5 :  return 0.05
    if epoch < 15 : return 0.12

    return 0.08

* Bu fonksiyon hangi epoch değerlerinde hangi drop_var oranına sahip olacağını belirler.Yani 3. epochta yüzde 5 inceleme , 12. epochda 0.12 inceleme devreye girecek.Eğer epoch sayısı if e giremezse yüzde 8 lik bir işlem uygulanacak.

In [ ]:
class UnkSchedule(tf.keras.callbacks.Callback):
    def on_epoch_begin(self ,epoch ,  logs=None):
        rate = schedule(epoch)
        drop_var.assign(rate)
        print(f"[UNK-Dropout] epoch {epoch} → rate={rate:.2f}")



* Burada bir callbacks haline getirdik.Aslında dataset pipeline ı gibi bir işlem bu.Nasılsa EarlyStopping bir callbacks ise , bu da bir callbacks haline geldi.Yukarıda anlatıldığı gibi işleme soktuk.

* Orada bulunan logs parametresi şunu ifade eder:

---- Biz loss fonksiyonu çağırırken val_loss gibi değişkenler ortaya çıkıyor.Aslında bu logs değişkeni yanındaki logları ifade ediyor. :d


-----
-----

### -- Bir kaç soru cevap ekleyelim.İlk öğrendiğimde bana zor gelen bir kaç soru üzerinden sizinle sohbet edelim. -- 

----

### 1) Geçici Tokenizer Neden Oluşturuldu?  
**(Hangi Kontrolleri Yapıyor, Bize Nasıl Fayda Sağlıyor?)**

1. **Frekans Sayımı İçin Tek Geçiş**  
   - Korpustaki **tüm kelimeleri** tarar,  
   - `word_counts = {token: görüldüğü_sayısı}` sözlüğü üretir.

2. **`min_freq` Filtresi Uygulamak**  
   - `min_freq` altındaki “çok nadir” kelimeleri sözlük dışı bırakır.  
   - Gürültüyü azaltır, bellek kullanımını düşürür.

3. **Özel Token’ları Garantiye Almak**  
   - `<user>`, `<bot>`, `<casual>`, `<unk>`, `<sos>`, `<eos>`  
     frekansa bakılmaksızın `keep_tokens` listesine eklenir.  

4. **`max_vocab` Kesiti (İsteğe Bağlı)**  
   - En yüksek frekanslı ilk `N` kelimeyi tutarak sözlüğü sınırlar.  
   - Embedding tablosu küçük kalır, model hızı artar.

> Eğer geçici tokenizer yerine doğrudan `Tokenizer.fit_on_texts` yapsaydık:  
> - **Tüm nadir kelimeler** sözlüğe girerdi → Vocab şişerdi.  
> - “UNK fırlaması” sonrası sorunlar için geriye dönüp temizleme gerekir, verimsiz olur.

Kısacası: **Geçici tokenizer**, kalıcı sözlüğü “temiz & kontrollü” inşa etmek için ara toplama ve filtreleme katmanıdır.


### 2) `keep_tokens` Değişkeni Ne İşe Yarar?  
**(Tokenizer İçindeki Rolü)**

`keep_tokens`, korpusta tutmak istediğimiz **“geçerli”** kelimelerin listesidir.  
Bu liste iki aşamalı filtrelemenin sonucunda oluşur:

1. **`min_freq` Filtresi**  
   - `word_counts[token] ≥ min_freq` olan kelimeler seçilir.  
   - Böylece tek-tük (çok nadir) kelimeler elenir → sözlük şişmez.

2. **Özel Token Zorlaması**  
   - `<user>`, `<bot>`, `<casual>`, `<unk>`, `<sos>`, `<eos>`  
     frekansa bakılmaksızın listeye eklenir.  
   - Model, rol/stil ve kontrol sembollerini kesinlikle tanır.

> **Sonuç:**  
> - `keep_tokens` **yalnızca** sözlükte yer alacak kelimeleri içerir.  
> - Nihai `Tokenizer.fit_on_texts(keep_tokens)` çağrısıyla **kelime-ID haritası** sabitlenir.  
> - Geri kalan tüm nadir kelimeler eğitim sırasında **`<unk>`** olarak temsil edilir, model gereksiz gürültüyle uğraşmaz.


### 3) `MAX_VOCAB` Parametresi ve Sıralama Fonksiyonunun Amacı  

`MAX_VOCAB` ≈ “sözlük için **üst sınır**”.  
Kodda yapılan işlem:

```python
if MAX_VOCAB:
    keep_tokens = sorted(
        keep_tokens,
        key=lambda w: word_counts.get(w, 0),  # en sık kelimeler öne
        reverse=True
    )[:MAX_VOCAB]                             # ilk N kelimeyi tut


### 4-) Kelime kapsamı ( coverage) neden kontrol ediliyor? %100 değişkeni ya da min_freq'in oradaki görevi nedir ?

* Kelime kapsamı tokenizerın kelime-kelime verinin yüzde kaçını kapsadığına bakıyor.Daha açıklayıcı bir tanım yapmak istersek ;

-- Min_freq = 1 iken % 100

-- Min_freq = 2 iken %96

-- Min_freq = 6 iken %67 lik bir kapsam ortaya çıktığını varsayalım.

* Frekans değerini 1 olarak ayarladığımız zaman ( her kelime 1 kere geçiyorsa ) modelin tamamına eşleştirme yapabiliyoruz.Aynı şekilde frekans değerini 6 yaparsak , tokenizer ( 6 kere geçen kelimeler üzerinden ) %67 lik bir kapsam oluşturabiliyor.Aslında burada temel prensibe gelecek olursak: 

##### UNK ya da OOV tokenını modele mantıklı bir şekilde öğretmek.